<a href="https://colab.research.google.com/github/YUCAVALCANTE/Tutorials_Linkedin/blob/main/Polars.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exemplo Fácil: A Beleza da Simplicidade

Vamos começar com algo leve e direto. Imagine que você está organizando uma pequena reunião e precisa saber rapidamente quais dos seus convidados são adultos com mais de 29 anos para um brinde especial. Com Polars, essa informação está ao alcance de um comando, sem complicações nem demoras.


In [1]:
import polars as pl

data = {
    "nome": ["Alice", "Bob", "Charlie"],
    "idade": [25, 30, 35],
    "cidade": ["New York", "Los Angeles", "Chicago"]
}

df = pl.DataFrame(data)
resultado = df.filter(pl.col("idade") > 29)
print(resultado)


shape: (2, 3)
┌─────────┬───────┬─────────────┐
│ nome    ┆ idade ┆ cidade      │
│ ---     ┆ ---   ┆ ---         │
│ str     ┆ i64   ┆ str         │
╞═════════╪═══════╪═════════════╡
│ Bob     ┆ 30    ┆ Los Angeles │
│ Charlie ┆ 35    ┆ Chicago     │
└─────────┴───────┴─────────────┘


## Exemplo Complexo: O Poder do Polars em Grandes Conjuntos de Dados

Imagine que você é um analista de dados em uma grande empresa e precisa processar **milhões de registros de transações** para identificar **tendências de consumo**. Seu desafio é:
- Filtrar apenas as transações **acima de um determinado valor**.
- Agrupar os dados por **categoria** e calcular o **total movimentado**.
- Ordenar os resultados para destacar **as categorias com maior volume de transações**.

Dado o tamanho dos dados, escolher a ferramenta certa é fundamental. Vamos comparar como **Pandas** e **Polars** lidam com essa tarefa e entender **por que o Polars é mais eficiente** para processar grandes volumes de informações.



In [6]:
import pandas as pd
import polars as pl
import numpy as np
import time

# Gerando um dataset com 10 milhões de transações
N = 10_000_000

np.random.seed(42)
categorias = ["Alimentação", "Transporte", "Saúde", "Lazer", "Educação", "Tecnologia"]
dados = {
    "id": np.arange(N),
    "categoria": np.random.choice(categorias, N),
    "valor": np.random.uniform(5, 1000, N)
}

# Criando DataFrames em Pandas e Polars
df_pandas = pd.DataFrame(dados)
df_polars = pl.DataFrame(dados)

print("Dataset gerado com 10 milhões de registros!")


Dataset gerado com 10 milhões de registros!


In [7]:
# Medindo tempo de execução no Pandas
start = time.time()

df_pandas_filtrado = df_pandas[df_pandas["valor"] > 500]  # Filtrando valores acima de 500
df_pandas_agrupado = df_pandas_filtrado.groupby("categoria")["valor"].sum().reset_index()  # Agrupando por categoria
df_pandas_agrupado = df_pandas_agrupado.sort_values("valor", ascending=False)  # Ordenando pelo total movimentado

end = time.time()
print(f"Tempo de execução no Pandas: {end - start:.2f} segundos")
print(df_pandas_agrupado)


Tempo de execução no Pandas: 1.10 segundos
     categoria         valor
2        Lazer  6.287907e+08
4   Tecnologia  6.285292e+08
3        Saúde  6.284228e+08
5   Transporte  6.283195e+08
1     Educação  6.282658e+08
0  Alimentação  6.275208e+08


In [8]:
# Medindo tempo de execução no Polars
start = time.time()

df_polars_resultado = (
    df_polars.lazy()  # Ativando Lazy Execution
    .filter(pl.col("valor") > 500)  # Filtrando valores acima de 500
    .group_by("categoria")  # Agrupando por categoria
    .agg(pl.col("valor").sum().alias("total_movimentado"))  # Somando os valores
    .sort("total_movimentado", descending=True)  # Ordenando pelo total movimentado
    .collect()  # Executando a query
)

end = time.time()
print(f"Tempo de execução no Polars: {end - start:.2f} segundos")
print(df_polars_resultado)


Tempo de execução no Polars: 0.51 segundos
shape: (6, 2)
┌─────────────┬───────────────────┐
│ categoria   ┆ total_movimentado │
│ ---         ┆ ---               │
│ str         ┆ f64               │
╞═════════════╪═══════════════════╡
│ Lazer       ┆ 6.2879e8          │
│ Tecnologia  ┆ 6.2853e8          │
│ Saúde       ┆ 6.2842e8          │
│ Transporte  ┆ 6.2832e8          │
│ Educação    ┆ 6.2827e8          │
│ Alimentação ┆ 6.2752e8          │
└─────────────┴───────────────────┘
